In [6]:
import os
import numpy as np
import random
import scipy as sp
from matplotlib import pyplot as plt
import seaborn as sns
import itertools
import copy
import pandas as pd

In [7]:
def generate_sched(stages, transition_proba, length=15):
    """_summary_ To generate a trace
    """
    res = []
    prev = random.choice(stages)
    res.append((0, prev))
    
    for i in range(1, length):
        prev = random.choices(stages, transition_proba[prev], k=1)[0]
        res.append((i, prev))
    return res

def gen_job(stages): 
    return {i: None for i in stages}

def count_done(works, stages):
    res = 0
    latencies = []
    first_stage = stages[0]
    last_stage = stages[-1]
    while works[res][last_stage] is not None:
        w = works[res]
        latencies.append(w[last_stage] - w[first_stage] + 1)
        res += 1
    return res, latencies

In [59]:
def run_simulation_(stages, transitions, total_time, n_cores):
    trace = [generate_sched(stages=stages, transition_proba=transitions, length=total_time) for _ in range(n_cores)]
    lasts = [0 for _ in stages]
    todo = [gen_job(stages) for _ in range(total_time * n_cores // 2)]
    for job in todo:
        job['last_update'] = -1
    hits = [[False for _ in range(total_time)] for _ in range(n_cores)]
    
    
    for timestamp, workers in enumerate(zip(*trace)):
        workers = list(workers)
        workers.sort(key=lambda x: x[1])
        
        for core, w in enumerate(workers):
            index, worker = w
            if worker == stages[0]:
                hits[core][index] = True
                if todo[lasts[worker]][worker] is None:
                    todo[lasts[worker]][worker] = (core, index, worker)
                    todo[lasts[worker]]['last_update'] = timestamp
                    lasts[worker] += 1
                else:
                    lasts[worker] += 1
                    todo[lasts[worker]][worker] = (core, index, worker)
                    todo[lasts[worker]]['last_update'] = timestamp
            else:
                if todo[lasts[worker]][worker-1] is None or todo[lasts[worker]]['last_update'] == timestamp:
                    continue
                else:
                    hits[core][index] = True
                    if todo[lasts[worker]][worker] is None:
                        todo[lasts[worker]][worker] = (core, index, worker)
                        todo[lasts[worker]]['last_update'] = timestamp
                        lasts[worker] += 1
                    else:
                        lasts[worker] += 1
                        todo[lasts[worker]][worker] = (core, index, worker)
                        todo[lasts[worker]]['last_update'] = timestamp
    
    return trace, todo, hits
#     total_work, latencies = count_done(todo, stages)

#     throughput = total_work / total_time
#     tail_latency = np.percentile(latencies, 99)
#     mean_latency = np.mean(latencies)
#     median_latency = np.median(latencies)

#     return throughput, median_latency, mean_latency, tail_latency

def run_simulation(stages, transitions, total_time, n_cores, runs=1):
    names = ['throughput', 'median_latency', 'mean_latency', 'tail_latency']
    data = [run_simulation_(stages, transitions, total_time, n_cores) for _ in range(runs)]
    return pd.DataFrame(data=data, columns=names)

In [60]:
def rand(x, y, n):
    if x == y:
        return 0
    else:
        return 1/(n-1)

def ordered(x, y, n, weight=1):
    if x == y:
        return 0
    m = y - x
    if m < 0:
        m += n
    if m == 1:
        return weight
    else:
        return (1-weight)/(n-2)

def generate_order_transition(weight, stages):
    n_stages = len(stages)
    res = [ [0 for _ in stages ] for _ in stages ]
    
    for x, y in itertools.product(stages, stages):
        res[x][y] = ordered(x, y, n_stages, weight=weight)
    return res

In [61]:
n_stages = 3
stages = [i for i in range(n_stages)]
stages_completion = [10, 30, 20]
skip_cost = 2
n_cores = 4
total_time = 1000
proba = [ [0 for _ in stages] for _ in stages ]
random_transitions = [
    [0, .5, .5],
    [.5, 0, .5],
    [.5, .5, 0]
]
ordered_transitions = [
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
]
semi_ordered_transitions = [
    [0, .99, .01],
    [.01, 0, .99],
    [.99, .01, 0]
]

rand_weights = [.50, .75, .90, .99, 1]

In [62]:
random_transitions = copy.deepcopy(proba)
order_transition = {i: generate_order_transition(i, stages) for i in rand_weights}

for x, y in itertools.product(stages, stages):
    random_transitions[x][y] = rand(x, y, n_stages)

In [63]:
def order_transition(i):
    return generate_order_transition(i, stages)

In [83]:
def count_done_jobs(todo, stages):
    return sum(map(lambda x: 1 if check_completed(x, stages) else 0, todo))

In [64]:
print("Optimal throughput:", n_cores/n_stages)
print("Optimal latency:", n_stages)

Optimal throughput: 1.3333333333333333
Optimal latency: 3


In [65]:
traces, todo, hits = run_simulation_(stages, random_transitions, total_time, n_cores)

In [67]:
completion_times = {s: c for s, c in zip(stages, stages_completion)}
# completion_times

In [72]:
completions = [ [None for _ in range(total_time)] for _ in range(n_cores) ]

for index, core in itertools.product(range(total_time), range(n_cores)):
    if hits[core][index]:
        completions[core][index] = completion_times[traces[core][index][1]]
    else:
        completions[core][index] = skip_cost

In [78]:
def check_completed(job, stages):
    for i in stages:
        if job[i] is None:
            return False
    return True

In [79]:
todo[:10]

[{0: (0, 0, 0), 1: (3, 1, 1), 2: (3, 2, 2), 'last_update': 2},
 {0: (0, 1, 0), 1: (0, 2, 1), 2: (1, 3, 2), 'last_update': 3},
 {0: (1, 1, 0), 1: (1, 2, 1), 2: (2, 3, 2), 'last_update': 3},
 {0: (2, 1, 0), 1: (2, 2, 1), 2: (3, 3, 2), 'last_update': 3},
 {0: (0, 3, 0), 1: (2, 4, 1), 2: (3, 5, 2), 'last_update': 5},
 {0: (0, 4, 0), 1: (1, 5, 1), 2: (2, 6, 2), 'last_update': 6},
 {0: (1, 4, 0), 1: (2, 5, 1), 2: (3, 6, 2), 'last_update': 6},
 {0: (0, 5, 0), 1: (1, 6, 1), 2: (3, 7, 2), 'last_update': 7},
 {0: (0, 6, 0), 1: (2, 7, 1), 2: (3, 9, 2), 'last_update': 9},
 {0: (0, 7, 0), 1: (1, 8, 1), 2: (2, 10, 2), 'last_update': 10}]

In [87]:
starts = []
ends = []

for core_line in completions:
    core_starts = []
    core_ends = []
    last_start = 0
    
    for job in core_line:
        core_starts.append(last_start)
        last_start += job
        core_ends.append(last_start)
    
    starts.append(core_starts)
    ends.append(core_ends)

In [96]:
def compute_latencies(todo, starts, ends, stages):
    latencies = []
    n_stages = len(stages)

    def compute_latency(job):
        if check_completed(job, stages):
            first = job[0]
            last = job[n_stages-1]
            start = starts[first[0]][first[1]]
            end = ends[last[0]][last[1]]
            return end - start
    
    for job in todo:
        latencies.append(compute_latency(job))
    return latencies

In [97]:
compute_latencies(todo, starts, ends, stages)

[42,
 42,
 60,
 60,
 34,
 62,
 52,
 54,
 46,
 82,
 54,
 46,
 72,
 100,
 64,
 66,
 66,
 104,
 130,
 102,
 36,
 16,
 62,
 54,
 -2,
 -2,
 26,
 -10,
 94,
 -38,
 148,
 186,
 56,
 -50,
 176,
 66,
 -88,
 -46,
 -38,
 96,
 -76,
 86,
 -66,
 184,
 112,
 86,
 -114,
 90,
 100,
 -74,
 -54,
 120,
 120,
 -64,
 -16,
 -44,
 92,
 56,
 114,
 -112,
 84,
 -48,
 -92,
 -44,
 136,
 36,
 -122,
 -140,
 -2,
 -150,
 -158,
 0,
 -140,
 -176,
 40,
 -196,
 10,
 -206,
 -38,
 -168,
 -214,
 -210,
 -210,
 0,
 6,
 -100,
 150,
 -198,
 -196,
 166,
 -196,
 36,
 -206,
 26,
 -96,
 130,
 112,
 90,
 -264,
 -264,
 114,
 -110,
 -242,
 -42,
 -104,
 -232,
 -104,
 -222,
 126,
 -212,
 -222,
 -74,
 -192,
 28,
 -74,
 -192,
 116,
 -192,
 -74,
 -192,
 -64,
 58,
 -64,
 -162,
 -24,
 102,
 98,
 -54,
 -152,
 -44,
 142,
 128,
 -44,
 -10,
 -122,
 158,
 -142,
 -14,
 -72,
 202,
 306,
 -62,
 238,
 -72,
 46,
 -12,
 278,
 -62,
 346,
 60,
 248,
 -62,
 336,
 -52,
 218,
 36,
 -42,
 56,
 -22,
 96,
 18,
 -22,
 320,
 238,
 66,
 412,
 584,
 228,
 76,
 360,


In [73]:
df_semi_ordered = run_simulation(stages, random_transitions, total_time, n_cores, runs=1000)
df_semi_ordered.median()

throughput         7.824000
median_latency    17.000000
mean_latency      16.421812
tail_latency      26.000000
dtype: float64

In [74]:
df_random = run_simulation(stages, order_transition(1), total_time, n_cores, runs=1000)
df_random.median()

throughput        7.976
median_latency    4.000
mean_latency      4.000
tail_latency      4.000
dtype: float64

In [75]:
df_random = run_simulation(stages, order_transition(.5), total_time, n_cores, runs=1000)
df_random.median()

throughput         7.832000
median_latency    16.000000
mean_latency      15.865836
tail_latency      25.000000
dtype: float64

In [76]:
df_random = run_simulation(stages, order_transition(.9), total_time, n_cores, runs=1000)
df_random.median()

throughput         7.906000
median_latency    10.000000
mean_latency       9.697151
tail_latency      14.000000
dtype: float64

In [77]:
df_random = run_simulation(stages, order_transition(.99), total_time, n_cores, runs=1000)
df_random.median()

throughput        7.954000
median_latency    6.000000
mean_latency      5.678755
tail_latency      8.000000
dtype: float64

In [78]:
df_random = run_simulation(stages, order_transition(.999), total_time, n_cores, runs=1000)
df_random.median()

throughput        7.970000
median_latency    4.000000
mean_latency      4.455207
tail_latency      5.500000
dtype: float64

In [79]:
df_random = run_simulation(stages, order_transition(.3), total_time, n_cores, runs=1000)
df_random.median()

throughput         7.822000
median_latency    16.000000
mean_latency      16.357977
tail_latency      26.000000
dtype: float64

In [80]:
df_random = run_simulation(stages, order_transition(.2), total_time, n_cores, runs=1000)
df_random.median()

throughput         7.823000
median_latency    17.000000
mean_latency      16.837681
tail_latency      26.000000
dtype: float64

In [81]:
df_random = run_simulation(stages, order_transition(.1), total_time, n_cores, runs=1000)
df_random.median()

throughput         7.819000
median_latency    17.000000
mean_latency      16.867546
tail_latency      26.000000
dtype: float64